<a href="https://colab.research.google.com/github/Emerson-Foco/Chat_Service_Evaluation_AI_Gemini_API/blob/main/Chat_Service_Evaluation_AI_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Steps

- [ ] Install and import Google SDK

In [3]:
!pip install -q -U google-generativeai

- [ ] Import Goodle SDK

In [4]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

- [ ] Setting your API key by *userdata.get*

In [5]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key)

# Setting your employee (AI)

> Adicionar aspas



- [ ] Employee info

In [ ]:
employee_name = 'Raquel'
employee_position = 'atendente'
employee_company = 'Google'
employee_products = 'AI Gemini'
employee_mood = 'amigavel e objetivo'

employee_info = {f'Você se chama {employee_name} e é {employee_position} de chat e trabalha para empresa {employee_company} '
                  f'com os produtos {employee_products}, realize um atendimento natural, humanizado e '
                  'sem assustar o cliente. Você não pode mentir e nem inventar.'
                  f' O que você não souber, diga que não possui a informação. Você é um atendente {employee_mood}!'}

print(employee_info)

- [ ] Employee model settings

In [48]:
employee_generation_config = {
    'candidate_count': 1,
    'temperature': 0.9,
}

employee_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

employee_model = genai.GenerativeModel('gemini-pro', generation_config=employee_generation_config, safety_settings=employee_safety_settings)

#Setting customer (AI)

- [ ] Customer info

In [ ]:
customer_name = 'Paulo'
customer_position = 'CEO'
customer_company = 'Alura'
customer_interest = 'IA para ajudar os alunos da sua plataforma de cursos online'
customer_mood = 'direto e objetivo' #experimente "furioso" e veja a avaliação do chat após a nova conversa.

customer_info = {f'Você é {customer_name}, {customer_position} da empresa {customer_company} e'
                  f' Você tem interesse em {customer_interest}. Você é {customer_mood}!'}

print(customer_info)

- [ ] Customer model settings

In [50]:
customer_generation_config = {
    'candidate_count': 1,
    'temperature': 0.9,
}

customer_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

customer_model = genai.GenerativeModel('gemini-pro', generation_config=customer_generation_config, safety_settings=customer_safety_settings)

# Setting chat evaluator (AI)

- [ ] Evaluator info

In [ ]:
#O objetivo é que o avaliador possa classificar as conversas para que o programa possa direcionar apenas conversas de interesse da empresa para análise.
evaluator_status_none = 'Se a conversa foi frutifera ou não teve nada de anormal, considere a conversa como "NONE".'
evaluator_status_warning = 'Se encontrar uma coversa que representa falhas de comunicação ou sem sentido, considere a conversa como "WARNING".'
evaluator_status_danger = 'Se encontrar uma conversa com ofensas ou ameaças, considere a conversa como "DANGER".'


evaluator_info = {'Você está monitorando conversas para garantir a qualidade de atendimento. Com isso, você tem os seguintes status: \n'
                  f'- {evaluator_status_none} \n- {evaluator_status_warning} \n- {evaluator_status_danger} \n'
                  'Com base nessas informações, responda qual status foi considerado na seguinte conversa: \n'}

print(evaluator_info)

- [ ] Evaluator model settings

In [52]:
evaluator_generation_config = {
    'candidate_count': 1,
    'temperature': 0,
}

evaluator_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

evaluator_model = genai.GenerativeModel('gemini-pro', generation_config=evaluator_generation_config, safety_settings=evaluator_safety_settings)

def chat_evaluation(chat):
  evaluation = evaluator_model.generate_content(f'{evaluator_info} {chat}')
  print(evaluation.text)

# Chat: AI vs AI

- [ ] Chat config

In [19]:
employee_chat = employee_model.start_chat(history=[])
customer_chat = customer_model.start_chat(history=[])

- [ ] Chat simulation

In [ ]:
employee_chat.history = []
customer_chat.history = []

prompt = ''
chat = ''

employee = ''
customer = customer_chat.send_message(f'{customer_info}\nInicie a conversa com apenas sua mensagem, sem precisar especificar "Você: olá"')

while prompt != 'stop':
  print('customer: ', customer.text, '\n')
  chat += customer.text
  employee = employee_chat.send_message(f'{employee_info} {customer.text}')
  print('employee: ', employee.text, '\n')
  chat += employee.text
  customer = customer_chat.send_message(f'{customer_info} {employee.text}')

  prompt = input('digite "stop" para parar ou enter para continuar: ')

- [ ] Chat evaluation

In [ ]:
#Analise a ultima conversa gravada para ela receber um status.
chat_evaluation(chat)